<a href="https://colab.research.google.com/github/vamshi2011/House-price-prediction-in-USA/blob/main/House_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/content/data.csv')
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null   int64  
 9   condition      4600 non-null   int64  
 10  sqft_above     4600 non-null   int64  
 11  sqft_basement  4600 non-null   int64  
 12  yr_built       4600 non-null   int64  
 13  yr_renovated   4600 non-null   int64  
 14  street         4600 non-null   object 
 15  city           4600 non-null   object 
 16  statezip       4600 non-null   object 
 17  country        4600 non-null   object 
dtypes: float

In [4]:
df = df.drop(columns=['date'], axis=1)

In [5]:
columns = df.select_dtypes(include='number')

In [6]:
columns.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992


In [7]:
for col in columns:
  q1 = columns[col].quantile(0.25)
  q3 = columns[col].quantile(0.75)
  iqr = q3 - q1
  upper = q3 + 1.5 * iqr
  lower = q1 - 1.5 * iqr

  df = df[(df[col] >= lower) & (df[col] <= upper)]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3448 entries, 0 to 4599
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          3448 non-null   float64
 1   bedrooms       3448 non-null   float64
 2   bathrooms      3448 non-null   float64
 3   sqft_living    3448 non-null   int64  
 4   sqft_lot       3448 non-null   int64  
 5   floors         3448 non-null   float64
 6   waterfront     3448 non-null   int64  
 7   view           3448 non-null   int64  
 8   condition      3448 non-null   int64  
 9   sqft_above     3448 non-null   int64  
 10  sqft_basement  3448 non-null   int64  
 11  yr_built       3448 non-null   int64  
 12  yr_renovated   3448 non-null   int64  
 13  street         3448 non-null   object 
 14  city           3448 non-null   object 
 15  statezip       3448 non-null   object 
 16  country        3448 non-null   object 
dtypes: float64(4), int64(9), object(4)
memory usage: 484.9+ K

In [9]:
cat_columns = df.select_dtypes(include='object').columns.to_list()
cat_columns

['street', 'city', 'statezip', 'country']

In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(df[cat_columns])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [11]:
encoded_columns = encoder.get_feature_names_out(cat_columns)
encoded_columns

array(['street_1 View Ln NE', 'street_10 W Etruria St',
       'street_100 20th Ave E', ..., 'statezip_WA 98288',
       'statezip_WA 98354', 'country_USA'], dtype=object)

In [12]:
df[encoded_columns] = encoder.fit_transform(df[cat_columns])

In [13]:
df.drop(columns=cat_columns, inplace=True)

In [14]:
df.head(3)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,...,statezip_WA 98166,statezip_WA 98168,statezip_WA 98177,statezip_WA 98178,statezip_WA 98188,statezip_WA 98198,statezip_WA 98199,statezip_WA 98288,statezip_WA 98354,country_USA
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
X = df.drop(columns=['price'])
y = df['price']

In [16]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,...,statezip_WA 98166,statezip_WA 98168,statezip_WA 98177,statezip_WA 98178,statezip_WA 98188,statezip_WA 98198,statezip_WA 98199,statezip_WA 98288,statezip_WA 98354,country_USA
0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,2.0,1.00,880,6380,1.0,0,0,3,880,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X[:] = scaler.fit_transform(X)

In [18]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,...,statezip_WA 98166,statezip_WA 98168,statezip_WA 98177,statezip_WA 98178,statezip_WA 98188,statezip_WA 98198,statezip_WA 98199,statezip_WA 98288,statezip_WA 98354,country_USA
0,0.333333,0.272727,0.213483,0.374295,0.2,0,0,0.333333,0.224852,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.333333,0.454545,0.379213,0.583156,0.0,0,0,0.666667,0.399408,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.333333,0.545455,0.398876,0.380403,0.0,0,0,0.666667,0.124260,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.666667,0.636364,0.382022,0.508256,0.0,0,0,0.666667,0.165680,0.533333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.090909,0.084270,0.294995,0.0,0,0,0.333333,0.088757,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
y.head()

,price
0,313000.0
2,342000.0
3,420000.0
4,550000.0
5,490000.0


In [20]:
y = y.to_numpy()
y = y.reshape(-1, 1)
y = scaler.fit_transform(y)

In [21]:
y

array([[0.27217391],
       [0.2973913 ],
       [0.36521739],
       ...,
       [0.36252536],
       [0.17686957],
       [0.19182609]])

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [24]:
y_pred = model.predict(X_test)

In [25]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
print("Mean Squared_error",mean_squared_error(y_test, y_pred))
print("Root mean squared error",root_mean_squared_error(y_test, y_pred))
print("Mean absolute error",mean_absolute_error(y_test, y_pred))

Mean Squared_error 2.427674811037581e+21
Root mean squared error 49271440115.32016
Mean absolute error 22035929967.75977
